In [2]:
import pandas as pd
df = pd.DataFrame.from_csv("promise_gzip.gz")

In [3]:
df.head(1)

,JavaSyntaxError,Source_code,code_col,code_line,code,astcode,label,repoSource,add,allow,...,new,npe,refactor,remov,report,set,support,test,use,database_source
0,True,"/**\n * Copyright 2015 Netflix, Inc.\n * \n * ...",NaN,NaN,"/**\n * Copyright 2015 Netflix, Inc.\n * \n * ...",NaN,p,ReactiveX-RxJava,1,0,...,0,0,0,0,0,0,0,0,0,Promise


In [4]:
df.isna().sum()

JavaSyntaxError      0
Source_code          0
code_col           542
code_line          542
code               230
astcode            312
label                0
repoSource           0
add                  0
allow                0
bug                  0
chang                0
error                0
fail                 0
fix                  0
implement            0
improv               0
issu                 0
method               0
new                  0
npe                  0
refactor             0
remov                0
report               0
set                  0
support              0
test                 0
use                  0
database_source      0
dtype: int64

In [5]:
df = df.dropna(subset=['astcode'])

In [6]:
df.isna().sum()

JavaSyntaxError      0
Source_code          0
code_col           230
code_line          230
code               230
astcode              0
label                0
repoSource           0
add                  0
allow                0
bug                  0
chang                0
error                0
fail                 0
fix                  0
implement            0
improv               0
issu                 0
method               0
new                  0
npe                  0
refactor             0
remov                0
report               0
set                  0
support              0
test                 0
use                  0
database_source      0
dtype: int64

In [6]:
from pd_doc2vec import doc2vec

x = doc2vec(df,'astcode', ['label'], build=True )
x.score(verbose=True)

label



Accuracy Score:  0.6222222222222222
Label Score:  [0.5898934398934399, 0.5676767676767678, 0.6124220300421672]


[[0.5898934398934399, 0.5676767676767678, 0.6124220300421672],
 0.7222222222222222]

In [62]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

models = []
thismax = 0
which_one=0

for i in tqdm(range(500)):
    # ("c" for corrective, "p" for perfective, "a" for adaptive
    from pd_doc2vec import doc2vec
    
    x = doc2vec(df,'astcode', ['label'], build=True )
    x.verbose = True
    models.append(x)
    df['result'] = x.predict(df['astcode'])


    labelaccuracy = f1_score(df['result'], df['label'], average=None)

    accuracy = accuracy_score(df['result'], df['label'])
    if accuracy>thismax:
        thismax = accuracy
        which_one = i
        print(accuracy, labelaccuracy)
    
print( max,which_one )

0.3739130434782609 [0.42285714 0.32608696 0.39330544]
0.37971014492753624 [0.375      0.35205993 0.41064639]
0.3884057971014493 [0.36879433 0.32846715 0.45818182]
0.39710144927536234 [0.53571429 0.32472325 0.38247012]
0.4 [0.41290323 0.34875445 0.4488189 ]
0.4028985507246377 [0.41333333 0.33823529 0.46268657]
0.4057971014492754 [0.42105263 0.33707865 0.46494465]
0.42028985507246375 [0.51282051 0.35164835 0.43678161]
0.4318840579710145 [0.41891892 0.33469388 0.51851852]
0.43768115942028984 [0.4875     0.34893617 0.48135593]
0.4492753623188406 [0.47852761 0.35684647 0.51048951]

<built-in function max> 397


In [63]:
x = models[397]
print(x)

In [31]:
models = []

In [33]:
print(x)

In [64]:
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle


with open('model.pkl', 'wb') as output:
    pickle.dump(x, output, pickle.HIGHEST_PROTOCOL)

In [35]:
print(df.columns.values)

['JavaSyntaxError' 'Source_code' 'code_col' 'code_line' 'code' 'astcode'
 'label' 'repoSource' 'add' 'allow' 'bug' 'chang' 'error' 'fail' 'fix'
 'implement' 'improv' 'issu' 'method' 'new' 'npe' 'refactor' 'remov'
 'report' 'set' 'support' 'test' 'use' 'database_source' 'result']


In [56]:
codes_to_try = []
for index, row in df.iterrows():
    if row['label'] == row['result']:
        codes_to_try.append(row['Source_code'])


In [59]:
def get_results(data):
    message = []
    code ={}
    import javalang
    import pandas as pd
    from pd_doc2vec import doc2vec
    try:
        import cPickle as pickle
    except ModuleNotFoundError:
        import pickle

    with open('model.pkl', 'rb') as input:
        x = pickle.load(input)
    try:
        tree = javalang.parse.parse(data)
        for codeblock in tree.children:
            result = x.predict_text_main(data)
            if result == "c":
                bug = "corrective"
            if result == "p":
                bug = "perfective"
            if result == "a":
                bug = "adaptive"

            try:
                line =   bug + " bug in CodeBlock at Line " + str(codeblock._position.line) + " Column " + str(codeblock._position.column) + " \n"
            except (AttributeError):
                pass
             #   line =  "Bug in CodeBlock at Line UNKNOWN  Column UNKOWN \n"
            message.append(line)


    except (javalang.parser.JavaSyntaxError, KeyError, javalang.tokenizer.LexerError) as e:
        message.append(str(e))
    code['msg'] = message
    print(message)


In [1]:
for i in codes_to_try:
    print(i)
    get_results(i)

NameError: name 'codes_to_try' is not defined